In [ ]:
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import theano.tensor as tt

In [ ]:
df = pd.read_csv('stats551-covid/data/ncs-pop-deaths-and-binary-mandates.csv', index_col=0)
df = df.rename(columns={"Age Group": "Age_Group", "COVID-19 Deaths": "covid_19_deaths"})
test_df = df[df["Month"]==7]
sex = np.array(test_df["Sex"])
mandate = test_df["Mandate"]
age = test_df["Age_Group"]
covid_deaths = test_df["covid_19_deaths"]
population = test_df["Population"]/1000000 # makes the population in units of millions
n = len(test_df["Age_Group"].unique()) # should decrease by 1 after proper age filtering

In [ ]:
pd.get_dummies(test_df["Age_Group"])

In [ ]:
age_data = pd.get_dummies(test_df["Age_Group"], drop_first=True)
sex_data = pd.get_dummies(test_df["Sex"], drop_first=True)
analysis_df = pd.concat([age_data, sex_data, mandate, covid_deaths, population], axis=1)
analysis_df

In [ ]:
# First Manual Attempt

# with pm.Model() as model:
    
# #     alpha = pm.DiscreteUniform('alpha', lower=0, upper=n, shape=len(age_data.columns))
# #     delta = pm.Normal('delta', mu=1, sigma=.1)
#     beta = pm.Normal('beta', mu=1, sigma=.1)
    
#     mean = pm.math.matrix_dot(mandate, beta)
#     print(type(mean))
    
#     obs = pm.Poisson('y_obs', mu=population*tt.exp(mean), observed=covid_deaths)
#     trace = pm.sample(1000)

In [ ]:
with pm.Model() as model:
    
    # Parameters I have excluded for now.
#     alpha = pm.DiscreteUniform('alpha', lower=0, upper=n, shape=len(age_data.columns))
#     delta = pm.Normal('delta', mu=1, sigma=.1)

    # spike and slab prior
    tau = pm.InverseGamma('tau', alpha=20, beta=20)
    xi = pm.Bernoulli('xi', p=0.8)
    beta = pm.Normal('beta', mu=0, sigma=tau)
    
    # mean setup for likelihood
    mandate = np.array(mandate).astype(theano.config.floatX)
    population = np.array(population).astype(theano.config.floatX)
    w = theano.shared(mandate, 'w')
    mean = pm.math.matrix_dot(w, xi*beta)
    
    # likelihood
    obs = pm.Poisson('y_obs', mu=population*tt.exp(mean), observed=covid_deaths)
    
    # sample from posterior
    trace = pm.sample(1000)

In [65]:
# Garbage Test Code

# import numpy as np
# import theano
# import theano.tensor as tt
# w_values = np.random.randn(1152, 10).astype(theano.config.floatX)
# input_values = np.random.randn(1152, 1).astype(theano.config.floatX)
# print(type(input_values))
# w = theano.shared(w_values, 'w')
# input = theano.shared(input_values, 'input')
# print(type(input))
# result = tt.exp(input.T)
# print(result.eval())

In [ ]:
# Manual Attempt

# with pm.Model() as model:
#     tau = pm.InverseGamma(r'$\tau$', alpha=20, beta=20)
#     Z = pm.Normal("Z", mu=0, sigma=1) # should these multivariate?
#     eps = pm.HalfCauchy(r'$\eps', beta=5)
#     Xi = pm.Bernoulli(r'$\Xi', p=0.8)
#     alpha = pm.DiscreteUniform(r'$\alpha$', lower=0, upper=n)
#     beta = Xi*Z
#     mean = age*alpha #+ pm.math.dot(sex,delta) + pm.math.dot(mandate, Xi)
    
    # obs = pm.Poisson('y', mu=population*mean, observed=covid_deaths)

In [ ]:
# # GLM Attempt

# with pm.Model() as model_glm:
#     priors = {
#         "age": pm.DiscreteUniform.dist(lower=0, upper=n, shape=15),
#         "Sex": pm.Bernoulli.dist(p=0.5, shape=2),
#         "Mandate": pm.Normal.dist(0, 0.05)
#     }
#     pm.GLM.from_formula("covid_19_deaths ~ .", test_df, family='poisson', priors=priors)
#     trace = pm.sample(2000, tune=1500)